***IMPORTS***

In [2]:
#Data
import pandas as pd
import numpy as np
import itertools
import pickle
import plotly as plt

#Processing
from sklearn.impute import KNNImputer
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

#Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

#Metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score

***DATA CLEANING***

**Analyse what the data represents and which variables have enough data to work with**

In [3]:
#Load_raw_data
rawdata =  pd.read_csv('C:/Users/Admiral Akhbar/Desktop/Markit/ds_assessment_data_2008_2016.csv', na_values=["inf","-inf",""," "], keep_default_na=True)

In [ ]:
#Set output options to explore data more easily
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 20)
pd.options.plotting.backend = "plotly"

In [ ]:
###Check out the data to see if it loaded ok
rawdata.head(10)
rawdata.shape
rawdata.describe()

In [ ]:
#Initial missing value check
missing_values = pd.DataFrame(rawdata.isnull().sum(),columns=['NA_count'])
missing_values['var'] = missing_values.index
missing_values.sort_values(by='NA_count',ascending=False,inplace=True)

In [ ]:
#Let's spend some time exploring the summary stats
summary_stats = pd.DataFrame(rawdata.describe())

From the summary stats we can see blah blah...
From the summary stats it's clear we have some infinite values in the raw data here
I have chosen to fix this by setting them as NA in the data read_csv

In [ ]:
# Now let's drop variables with a lot of missing values that are also highly correlated with variables we keep
# These vars are probably not worth imputing/fixing
# An alternative approach here would be to conisder dropping firms with lots of missing values across variables
# However, without knowing much about the firms in the data I believe this can bias the data and model results

In [ ]:
#We have 856 year-company observations in the dataset
#Let's first consider throwing away variables with less than say 2/3rds of year-company observations
#In this case I set the threshold at 350 obs
vars_to_drop = missing_values[missing_values['NA_count']>=350]['var']

len(vars_to_drop) #This gives us 62 candidates to drop

In [ ]:
#Let's set up the data as a panel dataset with id and year
rawdata.set_index(['id','fiscal_year_end_year'], inplace=True, verify_integrity=True)

#Let's get rid of calender year, we won't need it now as year is in the index
rawdata.drop(columns='calendar_end', inplace=True)

In [ ]:
#Now I find the highly correlated variables for the ones I want to throw out 
#I then decide if the variables I want to drop have sensible financial proxies left in the dataset
#There's probably a package that does this properly for panel data in Python but I struggled to find one, so did it by hand

In [ ]:
#Set up vars that we don't want to drop to check correlations 
vars_to_keep = rawdata.columns.to_series()
vars_to_keep = vars_to_keep[~vars_to_keep.isin(vars_to_drop)]

In [ ]:
#Set_up a full correlation matrix
cor_matrix_full = rawdata.corr()

#Set up Data frame for check
cor_matrix_check = pd.DataFrame(index=vars_to_drop)
cor_matrix_drop = cor_matrix_full.loc[vars_to_drop,vars_to_keep]

#Bring out index values for easy intrective sorting
cor_matrix_check["var_to_drop"] = cor_matrix_check.index.values

#Find max corr across vars
cor_matrix_check['Max_corr']=cor_matrix_drop.max(axis=1)

In [ ]:
#Find vars that have maxx corr
#Get variables to keep names
cor_matrix_drop = pd.melt(cor_matrix_drop, ignore_index=False, value_name='corr')

In [ ]:
#Leave only variables where correlation matches max cor
cor_matrix_check = pd.merge(cor_matrix_drop,cor_matrix_check,how='inner',left_index=True,right_index=True)
cor_matrix_check = cor_matrix_check.loc[cor_matrix_check['corr']==cor_matrix_check['Max_corr'],:]
print(cor_matrix_check)

In [ ]:
#Based on inspecting cor_matrix_check I decided not to drop the following variables:
# FCF
# as they/it have/has no sensible (from a financial analysis point of view) proxies left in the dataset
vars_to_drop = vars_to_drop.drop(['FCF'])
vars_to_keep = pd.concat([vars_to_keep,pd.Series(['FCF'])],ignore_index=True)

In [ ]:
#Let's drop the other vars and check missing values again
rawdata.drop(columns=vars_to_drop, inplace=True)

In [ ]:
#Second missing value check to check it worked correctly
missing_values = pd.DataFrame(rawdata.isnull().sum(),columns=['NA_count'])
missing_values.sort_values(by='NA_count',ascending=False,inplace=True)

**Fix Missing Values and decide Divident Cut threshold**

Becasue we're working with Balance sheet / P&L / Cash Flow Statement items we can't just backfill/pad missing values

I would try to impute missing values using MICE (Multiple Imputation by Chained Equations) and some sensible financial relationships

However, this will likely be industry-specific (and GAAP/IFRS specific) and with no industry variable here, no units (currency, million/billion) for many of the other variables, I'll need to do something simple and easy to understand like  K-Nearest Neighbours (KNN)

We are using all the data here, which is a bit of a cheat (I am doing this before the test/train split
and so test data info will "leak" in the train dataset)

However, it shouldn't matter when it comes to evaluating performance on the 2017 hold-out set and imputing the missing values during the test/train split with KNN can result in extra varience in the model performance which is difficult to control
Given that we have a lot imputing to do, I have chosen to do it pre test-train split

In [ ]:
#First off, let's fix the dividend return variable
#It's ok to pad with 0s here as missing values are due to time cut-off
clean_data = rawdata.copy()
clean_data['Dividend_returns'] = clean_data['Dividend_returns'].fillna(value=0)

In [ ]:
#Let's also remove the Dividents variable, since we're predicting a cut and can't use it in this scenario
clean_data.drop(columns='Dividend', inplace=True)

Now let's encode a target variable, with a given threshold

The threshold will depend on how I want to use the model, presumably to trade ahead of cut announcements 

In practice I would want to set a threshold based on how significant the market reaction to the announcement of the cut is,
for example share price performance in a X-day period after a cut of a given size

In [ ]:
#For now let's make a sensible guess
threshold = float(-0.15)

In [ ]:
#Gen cut var
clean_data['Div_Cut']=0
clean_data.loc[clean_data['Dividend_returns'] <= threshold,'Div_Cut']=1

In [ ]:
#Let's see how many firms had a cut at that threshold
check = clean_data[['Div_Cut']].groupby(axis='index', level=[0]).mean()
check.sort_values(by='Div_Cut', axis=0, ascending=False, inplace=True)
print(len(check[check['Div_Cut']>0])," out of ",len(check)," firms or ",
      round(len(check[check['Div_Cut']>0])/len(check),2)*100, "% of all firms had a cut at the ",
      threshold, " threshold")

In [ ]:
#Let's see a rough distribution (across years) of cuts by firm
#check.plot().show()

In [ ]:
#Let's see how cuts vary across years at this threshold
check = clean_data[['Div_Cut']].groupby(axis='index', level=[1]).sum()
check.sort_values(by='Div_Cut', axis=0, ascending=False)
#check.plot().show()

Ok this threshold seems reasonable enough to proceed with

Now I will use K-Nearest Neighbours imputation to fix missing values 

As far as I can see, this should work off the data.frame index and so should take into account patterns in both firm and year

In [ ]:
# I think it's good practiceto make all the key parameters for the imputer explicit here
imputer = KNNImputer(missing_values=np.nan, n_neighbors=2, weights='uniform', metric='nan_euclidean', copy=True, add_indicator=False)

In [ ]:
#impute
imputed_data = pd.DataFrame(imputer.fit_transform(clean_data))
#add back var names
imputed_data.columns = clean_data.columns
#add back index
imputed_data.index = clean_data.index

In [ ]:
#Now let's drop divident returns as a column as this will not be known in the prediction year
imputed_data.drop('Dividend_returns', axis=1, inplace=True)

In [ ]:
#Now let's add back the firm id as a variable
imputed_data["firm_id"] = imputed_data.index.get_level_values(0)
#and then remove it from the index
imputed_data.index = imputed_data.index.droplevel(0)

**Test / Train Split for model selection**

One way to do this is to backtest rolling windows for a few years of data for all firms. The problem is this will leaves me with an unballanced panel by firm as not all firms have data for all years since 2008

A second way to do this is to hold out data for some of the firms each time, but use all years
    
A third option is to use all data (2008-2015) for all firms to train and test on 2016, given that is the year closest to the hold-out year (2017)

I have chosen the third option because I expect the same firms to be included in the hold-out set and I can reduce overfitting by cross-validation

In [ ]:
# I want to test the models on all firms for 2016
# Because the data is already sorted by year and firm_id (as this was the index)
# We could take every 6th observation, if all firms had existed in at the start of the data in 2008
# Let's check
print("Avg years per firm are ", str(len(imputed_data) / len(imputed_data["firm_id"].unique())))

# Sadly not all firms have existed since 2008

In [ ]:
# Therefore I must take the 2016 observation for each firm for the X_test set
# and leave the rest for the X_train set which leaves us with a slighly unequal panel by firm

X_test = imputed_data.loc[2016,imputed_data.columns != 'Div_Cut']
y_test = imputed_data.loc[2016,'Div_Cut']
X_train = imputed_data.loc[range(2008,2016),imputed_data.columns != 'Div_Cut']
y_train = imputed_data.loc[range(2008,2016),'Div_Cut']

In [ ]:
#Check that nothing went wrong
print('Check that X and y are equal')
print(len(X_test)==len(y_test),"for X and y test")
print(len(X_train)==len(y_train),"for X and y train")

print('Check I split the entire set')
print(len(X_test)+len(X_train)==len(imputed_data))

**Data transformation for modelling**

In [ ]:
#Now I transform my training data

#Encode firm_id as dummy var
Firm_Encoder = OneHotEncoder(categories='auto', drop=None, sparse=True, handle_unknown='ignore')

In [ ]:
#Scale features that are not finanial_health_score or firm_id by centering around the mean
Values_Encoder = StandardScaler()

In [ ]:
#Select features to center
Values_features = ['ASSETS', 'BPS', 'CAPEX', 'CFPS', 'CF_FIN',
       'CF_INV', 'CF_OP', 'DEPR_AMORT', 'EBIT', 'EBITDA', 'EPS', 'EPS_GAAP',
       'EPS_NONGAAP', 'FCF', 'G_A_EXP', 'INT_EXP', 'NDT', 'NET', 'NETBG',
       'PTI', 'SALES', 'SH_EQUITY', 'CAPEX_returns', 'CF_OP_returns',
       'EBIT_returns', 'EBITDA_returns', 'EPS_GAAP_returns', 'NDT_returns',
       'NET_returns', 'NETBG_returns', 'PTI_returns', 'SH_EQUITY_returns',
       'CFPS_returns', 'EPS_returns', 'SALES_returns', 'payout_ratio',
       'earnings_cover', 'cashflow_cover', 'debt_factor', 'profitability']

#Check we have the right features
X_train.columns[~X_train.columns.isin(Values_features)]

In [ ]:
#Now we can apply the transformations to the data
#Let's set_up a transformer object

data_transformer = ColumnTransformer(
    transformers=[
        ('firm', Firm_Encoder,['firm_id']),
        ('Values_Enc_1',Values_Encoder, Values_features)
    ],remainder='passthrough')

In [ ]:
#Fit to the data and transform it
data_transformer.fit(X_train)
X_train_trans = data_transformer.transform(X_train)

***MODELLING***

In [ ]:
#I'll first try out a few reliable models to understand how they perform and which one shows the most promise
